In [139]:
import geopy.distance
from geopy.distance import geodesic

In [150]:
# Define the latitude and longitude of each city
lat_long = {
    'Arad': (46.1667, 21.3167),
    'Bucharest': (44.4167, 26.1000),
    'Craiova': (44.3333, 23.8167),
    'Drobeta': (44.6259, 22.6566),
    'Eforie': (44.0667, 28.6333),
    'Fagaras': (45.8416, 24.9730),
    'Giurgiu': (43.9037, 25.9699),
    'Hirsova': (44.6833, 27.9500),
    'Iasi': (47.1585, 27.6014),
    'Lugoj': (45.6904, 21.9033),
    'Neamt': (46.9283, 26.3705),
    'Oradea': (47.0553, 21.9214),
    'Pitesti': (44.8565, 24.8697),
    'Rimnicu Vilcea': (45.1042, 24.3758),
    'Sibiu': (45.7977, 24.1521),
    'Timisoara': (45.7489, 21.2087),
    'Urziceni': (44.7167, 26.6333),
    'Vaslui': (46.6333, 27.7333),
    'Zerind': (46.6225, 21.5174)
}


In [151]:
# Define the straight-line distance heuristic function
def heuristic(start, end):
    start_lat, start_long = lat_long[start]
    end_lat, end_long = lat_long[end]
    return math.sqrt((end_lat - start_lat) ** 2 + (end_long - start_long) ** 2)

In [152]:
class Node:
    def __init__(self, city, coordinates):
        self.city = city
        self.coordinates = coordinates
        self.neighbors = []
        self.g = float('inf')
        self.f = float('inf')
        self.parent = None

    def distance(self, other):
        lat1, lon1 = self.coordinates
        lat2, lon2 = other.coordinates
        distance = geopy.distance.distance((lat1, lon1), (lat2, lon2)).km
        return distance

In [153]:
# A* search algorithm
def astar(start, goal, nodes):
    # Initialize the starting node's g and f values to 0
    start.g = 0
    start.f = start.distance(goal)

    # Create the open and closed lists
    open_list = [start]
    closed_list = []

    while open_list:
        # Get the node with the lowest f score from the open list
        current = min(open_list, key=lambda node: node.f)

        # Check if the goal node is reached
        if current == goal:
            path = []
            while current:
                path.append(current.city)
                current = current.parent
            path.reverse()
            return path, goal.g

        # Move the current node from the open list to the closed list
        open_list.remove(current)
        closed_list.append(current)

        # Check all of the current node's neighbors
        for neighbor in current.neighbors:
            # Calculate the tentative g value for the neighbor
            tentative_g = current.g + current.distance(neighbor)

            if tentative_g < neighbor.g:
                # Update the neighbor's g score
                neighbor.g = tentative_g
                # Update the neighbor's f score
                neighbor.f = neighbor.g + neighbor.distance(goal)
                # Set the current node as the neighbor's parent
                neighbor.parent = current

                if neighbor not in open_list and neighbor not in closed_list:
                    # Add the neighbor to the open list
                    open_list.append(neighbor)


In [154]:
# Create the nodes on the map of Romania
nodes = {}
for city, coordinates in lat_long.items():
    nodes[city] = Node(city, coordinates)

In [155]:
connections = {
    'Arad': ['Zerind', 'Sibiu', 'Timisoara'],
    'Bucharest': ['Giurgiu', 'Pitesti', 'Urziceni', 'Fagaras'],
    'Craiova': ['Drobeta', 'Pitesti', 'Rimnicu Vilcea'],
    'Drobeta': ['Craiova'],
    'Eforie': ['Hirsova'],
    'Fagaras': ['Sibiu', 'Bucharest'],
    'Giurgiu': ['Bucharest'],
    'Hirsova': ['Eforie', 'Urziceni'],
    'Iasi': ['Vaslui', 'Neamt'],
    'Lugoj': ['Timisoara'],
    'Neamt': ['Iasi'],
    'Oradea': ['Zerind', 'Sibiu'],
    'Pitesti': ['Rimnicu Vilcea', 'Bucharest', 'Craiova'],
    'Rimnicu Vilcea': ['Sibiu', 'Pitesti', 'Craiova'],
    'Sibiu': ['Oradea', 'Arad', 'Fagaras', 'Rimnicu Vilcea'],
    'Timisoara': ['Arad', 'Lugoj'],
    'Urziceni': ['Bucharest', 'Hirsova', 'Vaslui'],
    'Vaslui': ['Iasi', 'Urziceni'],
    'Zerind': ['Arad', 'Oradea']
}


In [156]:
for city, neighbors in connections.items():
    nodes[city].neighbors = [nodes[neighbor] for neighbor in neighbors]


In [157]:
# Define the start and goal nodes
start_city = 'Arad'
goal_city = 'Bucharest'
# Find the start and goal nodes from the nodes dictionary
start_node = None
goal_node = None
for node in nodes.values():
    if node.city == start_city:
        start_node = node
    elif node.city == goal_city:
        goal_node = node
        break
# Check if start and goal nodes are found
if start_node is None:
    raise ValueError("Start city not found in the map.")
if goal_node is None:
    raise ValueError("Goal city not found in the map.")


In [158]:
# Execute the A* search algorithm
shortest_path, distance = astar(nodes[start_city], nodes[goal_city], nodes)
# Print the shortest path and its distance
print("Shortest path from", start_city, "to", goal_city, ":", shortest_path)
print("Distance:", distance)

Shortest path from Arad to Bucharest : ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
Distance: 459.39541803611655
